# Historical Average as Baseline method
This file compute the historical Average

In [32]:
import pandas as pd
import numpy as np
import argparse
import torch
from Neo4j import NeoDriver

Exception ignored in: <function NeoDriver.__del__ at 0x12c5f9c60>
Traceback (most recent call last):
  File "/Users/rdm-nico/VSCode_Projects/GA/Traffic_Prediction_Project/preprocessing/Neo4j.py", line 264, in __del__
    self.close()
  File "/Users/rdm-nico/VSCode_Projects/GA/Traffic_Prediction_Project/preprocessing/Neo4j.py", line 14, in close
    self.driver.close()
    ^^^^^^^^^^^
AttributeError: 'NeoDriver' object has no attribute 'driver'


In [3]:
options = {'neo4jURL': 'bolt://localhost:7687', 'neo4juser': 'neo4j', 'neo4jpwd': ''}
namespace_o = argparse.Namespace(**options)

<class 'argparse.Namespace'>
connection confirmed


In [64]:
def HA(options, epochs=10, train_split=0.8):
    greeter = NeoDriver(options)

    for epoch in range(epochs):
        # query the dbms
        query="""WITH toInteger( rand() * 24) as hour, [0,15,30,45][toInteger(rand() * 4)] as minute
            UNWIND range(1,31) as day
            MATCH(s:Sensor)-[:HAS]-(m) 
            WHERE m.start_time.day = day AND m.start_time.hour = hour AND m.start_time.minute = minute 
            RETURN collect(m.total_flow) as vector, s.id as id 
            ORDER BY id"""
        df = greeter.run_query(query)['vector']
        # move to np
        mat = np.stack(df.values)
        # split by column and convert to Tensor
        Train, Test = np.hsplit(mat, [int(train_split*mat.shape[1])])
        print(f'train len: {Train.shape}\t Test shape: {Test.shape}')


        pred = np.mean(Train,axis=1)
        label = [Test[:,i] for i  in range(Test.shape[1])]
        mask = np.not_equal(label, 0)
        mask = mask.astype(np.float32)
        mask /= np.mean(mask)
        mae = np.abs(np.subtract(pred, label)).astype(np.float32)
        wape = np.divide(np.sum(mae), np.sum(label))
        wape = np.nan_to_num(wape * mask)
        rmse = np.square(mae)
        mape = np.divide(mae, label)
        mae = np.nan_to_num(mae * mask)
        mae = np.mean(mae)
        rmse = np.nan_to_num(rmse * mask)
        rmse = np.sqrt(np.mean(rmse))
        mape = np.nan_to_num(mape * mask)
        mape = np.mean(mape)
        print(f'step {epoch}: mae:{mae}\t rmse:{rmse}\t mape:{mape}')
    

In [66]:
HA(namespace_o)

<class 'argparse.Namespace'>
connection confirmed
train len: (259, 24)	 Test shape: (259, 7)
step 0: mae:10.885066986083984	 rmse:29.31426239013672	 mape:0.2695874827900709


/var/folders/zk/y_05r4dd4q51x5hx9n0nwwk80000gn/T/ipykernel_5698/7231377.py:29: RuntimeWarning: divide by zero encountered in divide
  mape = np.divide(mae, label)
/var/folders/zk/y_05r4dd4q51x5hx9n0nwwk80000gn/T/ipykernel_5698/7231377.py:29: RuntimeWarning: invalid value encountered in divide
  mape = np.divide(mae, label)
/var/folders/zk/y_05r4dd4q51x5hx9n0nwwk80000gn/T/ipykernel_5698/7231377.py:34: RuntimeWarning: invalid value encountered in multiply
  mape = np.nan_to_num(mape * mask)


train len: (259, 24)	 Test shape: (259, 7)
step 1: mae:15.376298904418945	 rmse:52.13982009887695	 mape:0.5300085304655253
train len: (259, 24)	 Test shape: (259, 7)
step 2: mae:7.327831268310547	 rmse:17.270254135131836	 mape:0.5700726629528278
train len: (259, 24)	 Test shape: (259, 7)
step 3: mae:14.638362884521484	 rmse:32.91659164428711	 mape:0.3159422721321001
train len: (259, 24)	 Test shape: (259, 7)
step 4: mae:14.378923416137695	 rmse:43.62812805175781	 mape:0.46866937063489417
train len: (259, 24)	 Test shape: (259, 7)
step 5: mae:6.364428997039795	 rmse:12.738496780395508	 mape:0.40332747894792104
train len: (259, 24)	 Test shape: (259, 7)
step 6: mae:5.7970380783081055	 rmse:17.773740768432617	 mape:0.39325860078175656
train len: (259, 24)	 Test shape: (259, 7)
step 7: mae:14.451576232910156	 rmse:22.34711456298828	 mape:0.8343206304093447
train len: (259, 24)	 Test shape: (259, 7)
step 8: mae:12.308202743530273	 rmse:27.536136627197266	 mape:0.2917082478436749
train len: 